In [1]:
import opendatasets as od

# Download the dataset
od.download("https://www.kaggle.com/datasets/yasserhessein/the-kvasir-dataset")

Dataset URL: https://www.kaggle.com/datasets/yasserhessein/the-kvasir-dataset


100%|██████████| 2.32G/2.32G [00:36<00:00, 68.1MB/s]


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the paths for source, training, testing, and validation
source_path = "the-kvasir-dataset/kvasir-dataset-v2"
train_path = "data/training"
test_path = "data/testing"
validation_path = "data/validation"

# Define the split ratios
train_ratio = 0.7
test_ratio = 0.2
validation_ratio = 0.1

# Create the directories
for path in [train_path, test_path, validation_path]:
    os.makedirs(path, exist_ok=True)    
    
# Process each class
for class_name in os.listdir(source_path):
    class_dir = os.path.join(source_path, class_name)
    if os.path.isdir(class_dir):
        # List all images
        images = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        
        # Split the dataset
        train_val, test = train_test_split(images, test_size=test_ratio, random_state=42)
        train, val = train_test_split(train_val, test_size=validation_ratio/(train_ratio+validation_ratio), random_state=42)
        
        # Define a function to copy files
        def copy_files(filenames, dest_dir):
            os.makedirs(dest_dir, exist_ok=True)
            for f in filenames:
                shutil.copy(f, dest_dir)
                
        # Copy the files
        copy_files(train, os.path.join(train_path, class_name))
        copy_files(test, os.path.join(test_path, class_name))
        copy_files(val, os.path.join(validation_path, class_name))

# Delete the downloaded dataset
shutil.rmtree("the-kvasir-dataset")

In [3]:
import os

# Get the length of the training, testing, and validation datasets
train_path = "data/training/normal-z-line"
test_path = "data/testing/normal-z-line"
validation_path = "data/validation/normal-z-line"

print("Training: ", len(os.listdir(train_path)))
print("Testing: ", len(os.listdir(test_path)))
print("Validation: ", len(os.listdir(validation_path)))

Training:  699
Testing:  200
Validation:  101


In [4]:
import torch

# Import PyTorch and setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.__version__)
print(device)

2.4.0
cuda


In [6]:
from torchvision import datasets, transforms

train_path = "data/training"
test_path = "data/testing"

# Create simple transformations
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Use ImageFolder to create datasets
train_dataset = datasets.ImageFolder(root=train_path, 
                                     transform=data_transform,
                                     target_transform=None)

test_dataset = datasets.ImageFolder(root=test_path,
                                    transform = data_transform)

print(f"Train dataset:\n {train_dataset}\nTest dataset:\n {test_dataset}")

Train dataset:
 Dataset ImageFolder
    Number of datapoints: 5592
    Root location: data/training
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test dataset:
 Dataset ImageFolder
    Number of datapoints: 1600
    Root location: data/testing
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [7]:
# Get the class names as a dict
class_names = train_dataset.class_to_idx
print(class_names)

{'dyed-lifted-polyps': 0, 'dyed-resection-margins': 1, 'esophagitis': 2, 'normal-cecum': 3, 'normal-pylorus': 4, 'normal-z-line': 5, 'polyps': 6, 'ulcerative-colitis': 7}


In [8]:
# Check the length of the datasets
len(train_dataset), len(test_dataset)

(5592, 1600)

In [9]:
from torch.utils.data import DataLoader
import os

NUM_WORKERS = os.cpu_count()

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=32, 
                          num_workers=NUM_WORKERS,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=32,
                         num_workers=NUM_WORKERS,
                         shuffle=False)

print(train_loader)
print(test_loader)

In [11]:
%%writefile modular/data_setup.py
"""
Defines the functionality for creating PyTorch DataLoaders for the multi-class classification dataset.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir: str, 
                       test_dir: str, 
                       transform: transforms.Compose, 
                       batch_size: int, 
                       num_workers: int=NUM_WORKERS):
    """Takes in a training and testing directory path and turns them into PyTorch DataLoaders.

    Args:
        train_dir (str): Path to training directory.
        test_dir (str): Path to testing directory.
        transform (transforms.Compose): Torchvision transforms to apply to the datasets.
        batch_size (int): Number of samples per batch in each DataLoader.
        num_workers (_type_): Number of workers per DataLoader. Currently set to os.cpu_count().

    Returns:
        Tuple: Returns a tuple of (train_loader, test_loader, class_names). Where class_names is a dict of the target classes.
    """
    # Use ImageFolder to create datasets
    train_data = datasets.ImageFolder(root=train_dir,
                                      transform=transform)
    test_data = datasets.ImageFolder(root=test_dir,
                                     transform=transform)
    
    # Get the class names
    class_names = train_data.class_to_idx
    
    # Create DataLoaders
    train_loader = DataLoader(dataset=train_data, 
                              batch_size=batch_size, 
                              num_workers=num_workers, 
                              shuffle=True,
                              pin_memory=True)
    
    test_loader = DataLoader(dataset=test_data,
                             batch_size=batch_size,
                             num_workers=num_workers,
                             shuffle=False,
                             pin_memory=True)
    
    return train_loader, test_loader, class_names

Overwriting modular/data_setup.py


In [12]:
import torch
from torch import nn

# Define the baseline model
class BaseLine(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_shape, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, output_shape),
            nn.ReLU()
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layer_stack(x)

In [13]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate the model
torch.manual_seed(42)
model = BaseLine(input_shape=224*224*3, 
                 hidden_units=10, 
                 output_shape=len(train_dataset.classes)).to(device)
model

BaseLine(
  (layer_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=150528, out_features=10, bias=True)
    (2): ReLU()
    (3): Linear(in_features=10, out_features=8, bias=True)
    (4): ReLU()
  )
)

In [14]:
import torch

# Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_loader))

# Get a single images from the batch
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# Perform a forward pass
model.eval()
with torch.inference_mode():
    pred = model(img_single.to(device))

# Print out the conversion from logits, to pred probs, and pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 224, 224])

Output logits:
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1466, 0.0979, 0.3001]],
       device='cuda:0')

Output prediction probabilities:
tensor([[0.1161, 0.1161, 0.1161, 0.1161, 0.1161, 0.1345, 0.1281, 0.1568]],
       device='cuda:0')

Output prediction label:
tensor([7], device='cuda:0')

Actual label:
5


In [17]:
%%writefile modular/models/baseline_model.py
"""
Defines a PyTorch baseline model for multi-class classification.
"""
import torch
from torch import nn

class BaseLine(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        """Defines a simple feedforward neural network for multi-class classification.

        Args:
            input_shape (int): Number of input channels.
            hidden_units (int): Number of hidden units between layers.
            output_shape (int): Number of output units.
        """
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_shape, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, output_shape),
            nn.ReLU()
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layer_stack(x)

Writing modular/models/baseline_model.py


In [18]:
%%writefile modular/engine.py
"""
Defines functions for training and testing PyTorch models.
"""
from typing import Tuple, List, Dict

import torch

from tqdm.auto import tqdm

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Turns the model into training mode and then runs through all the required training steps.

    Args:
        model (torch.nn.Module): A PyTorch model to be trained.
        dataloader (torch.utils.data.DataLoader): A DataLoader object for the training data.
        loss_fn (torch.nn.Module): A PyTorch loss function to minimize.
        optimizer (torch.optim.Optimizer): A PyTorch optimizer to update the model weights.
        device (torch.device): A target device to send the data and model to.

    Returns:
        Tuple[float, float]: A tuple of the average loss and accuracy across all batches.
    """
    # Put the model in training mode
    model.train()
    
    #Setup the loss and accuracy
    train_loss, train_acc = 0, 0
    
    # Iterate over the data
    for batch, (X, y) in enumerate(dataloader):
        # Send the data to the device
        X, y = X.to(device), y.to(device)
        
        # Forward pass
        y_pred = model(X)
        
        # Get the predictions
        y_pred = model(X)
        
        # Calculate the loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)
        
    # Adjust the metrics and get the avg loss and accuracy across all batches
    tran_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Turns the model into evaluation mode and then runs through all the required testing steps.

    Args:
        model (torch.nn.Module): A PyTorch model to be tested.
        dataloader (torch.utils.data.DataLoader): A DataLoader object for the testing data.
        loss_fn (torch.nn.Module): A PyTorch loss function to minimize.
        device (torch.device): A target device to send the data and model to.

    Returns:
        Tuple[float, float]: A tuple of the average loss and accuracy across all batches.
    """
    # Put the model in evaluation mode
    model.eval()
    
    # Setup the loss and accuracy
    test_loss, test_acc = 0, 0
    
    # Turn on inference mode
    with torch.inference_mode():
        # Iterate over the data
        for batch, (X, y) in enumerate(dataloader):
            # Send the data to the device
            X, y = X.to(device), y.to(device)
        
            # Forward pass
            y_pred = model(X)
        
            # Calculate the loss
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()
        
            # Calculate and accumulate accuracy metric across all batches
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            test_acc += (y_pred_class == y).sum().item()/len(y_pred)
        
    # Adjust the metrics and get the avg loss and accuracy across all batches
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_loader: torch.utils.data.DataLoader,
          test_loader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          device: torch.device,
          epochs: int) -> Dict[str, List[float]]:
    """Passes a model through training and testing steps for a specified number of epochs.

    Args:
        model (torch.nn.Module): A PyTorch model to be trained and tested.
        train_loader (torch.utils.data.DataLoader): A DataLoader object for the training data.
        test_loader (torch.utils.data.DataLoader): A DataLoader object for the testing data.
        loss_fn (torch.nn.Module): A PyTorch loss function to minimize.
        optimizer (torch.optim.Optimizer): A PyTorch optimizer to update the model weights.
        device (torch.device): A target device to send the data and model to.
        epochs (int): The number of epochs to train the model for.

    Returns:
        Dict[str, List[float]]: A dictionary of lists containing the training and testing metrics.
    """
    # Setup a dict to store results
    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}
    
    # Train the model
    for epoch in tqdm(range(epochs)):
        # Perform a training step
        train_loss, train_acc = train_step(model, train_loader, loss_fn, optimizer, device)
        
        # Perform a testing step
        test_loss, test_acc = test_step(model, test_loader, loss_fn, device)
        
        # Append the metrics to the dict
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        # Print the metrics
        print(f"Epoch: {epoch+1}/{epochs} | Train Loss: {train_loss:.5f} | Train Acc: {train_acc:.5f} | Test Loss: {test_loss:.5f} | Test Acc: {test_acc:.5f}")
    
    # Return the results at the end of the epochs
    return results

Writing modular/engine.py


In [19]:
%%writefile modular/utils.py
"""
Defines functions that contain various utility functions for PyTorch model training and saving.
"""
from pathlib import Path

import torch

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str) -> None:
    """Save a PyTorch model to a specified directory.

    Args:
        model (torch.nn.Module): A PyTorch model to be saved.
        target_dir (str): The directory path to save the model.
        model_name (str): The name of the model file.
    """
    # Create the target directory
    Path(target_dir).mkdir(parents=True, exist_ok=True)
    
    # Create model save path
    assert model_name.endswith(".pt") or model_name.endswith(".pth"), "Model name must end with .pt or .pth"
    model_save_path = Path(target_dir) / model_name
    
    # Save the model
    print(f"Saving model to: {model_save_path}")
    torch.save(model.state_dict(), model_save_path)

Writing modular/utils.py


In [26]:
%%writefile modular/train.py
"""
Defines the training script for the PyTorch model.
"""
import os

import torch
from torchvision import transforms

import sys
sys.path.append("modular")
sys.path.append("modular/models")

import data_setup, engine, baseline_model

# Setup hyperparameters
NUM_EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001
HIDDEN_UNITS = 10

# Setup the directories
train_dir = "data/training"
test_dir = "data/testing"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Setup the transformations
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create the DataLoaders using data_setup.py
train_loader, test_loader, class_names = data_setup.create_dataloaders(train_dir, 
                                                                       test_dir, 
                                                                       data_transform, 
                                                                       BATCH_SIZE)

# Create the model
model = baseline_model.BaseLine(input_shape=224*224*3,
                                hidden_units=HIDDEN_UNITS,
                                output_shape=len(class_names)).to(device)

# Set the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Start training the model using engine.py
engine.train(model, train_loader, test_loader, loss_fn, optimizer, device, NUM_EPOCHS)

Overwriting modular/train.py
